In [6]:
import pandas as pd

df = pd.read_csv("../data/diabetic_data.csv")
print(df.shape)
print(df.head())


(101766, 50)
   encounter_id  patient_nbr             race  gender      age weight  \
0       2278392      8222157        Caucasian  Female   [0-10)      ?   
1        149190     55629189        Caucasian  Female  [10-20)      ?   
2         64410     86047875  AfricanAmerican  Female  [20-30)      ?   
3        500364     82442376        Caucasian    Male  [30-40)      ?   
4         16680     42519267        Caucasian    Male  [40-50)      ?   

   admission_type_id  discharge_disposition_id  admission_source_id  \
0                  6                        25                    1   
1                  1                         1                    7   
2                  1                         1                    7   
3                  1                         1                    7   
4                  1                         1                    7   

   time_in_hospital  ... citoglipton insulin  glyburide-metformin  \
0                 1  ...          No      No        

In [7]:
df['readmitted_30days'] = df['readmitted'].apply(lambda x: 1 if x == '<30' else 0)

In [8]:
df = df.drop(columns=['encounter_id','patient_nbr','readmitted'])

In [9]:
df = df.replace('?', pd.NA)

In [10]:
df = df.dropna(axis=1, thresh=0.5*len(df))  

In [11]:
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].fillna(df[col].mode()[0])  
    else:
        df[col] = df[col].fillna(df[col].median())  

In [12]:
cat_cols = df.select_dtypes(include=['object']).columns
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

In [13]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['readmitted_30days'])
y = df['readmitted_30days']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [14]:
df.to_csv("../dataclean/clean_data.csv", index=False)
